In [1]:
from langgraph.graph import END, StateGraph, add_messages
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
from typing import TypedDict, Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode


load_dotenv()

True

In [2]:
class ChatbotState(TypedDict):
    messages : Annotated[list, add_messages]

In [3]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.1, max_tokens=1000)
search_tool = TavilySearchResults()
tools = [search_tool]

llm_with_tools = llm.bind_tools(tools=tools)
# print(llm_with_tools.invoke("what is current weather in bangalore?"))

In [4]:
def chat_bot_node(state: ChatbotState) -> ChatbotState:
    """
    A simple chatbot node that appends a response to the messages in the state.
    
    Args:
        state (ChatbotState): The current state of the chatbot.
        
    Returns:
        ChatbotState: The updated state with the chatbot's response appended.
    """
    
    return {"messages": llm_with_tools.invoke(state['messages'])}

In [5]:
def tools_router(state: ChatbotState):
    last_message = state['messages'][-1]
    if (hasattr(last_message, 'tool_calls') and len(last_message.tool_calls) > 0):
        return "tool_node"
    else:
        return END

In [6]:
tool_node = ToolNode(tools = tools)

flow = StateGraph(ChatbotState)
flow.add_node("chat_bot", chat_bot_node)
flow.add_node("tool_node", tool_node)
flow.set_entry_point("chat_bot")
flow.add_conditional_edges("chat_bot", tools_router)
flow.add_edge("tool_node", "chat_bot")

app = flow.compile()
# Example usage
# print(app.invoke(ChatbotState(messages=[HumanMessage(content="What is the current weather in Bangalore?")])))

# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ["exit", "quit"]:
#         break
    
#     state = ChatbotState(messages=[HumanMessage(content=user_input)])
#     response = app.invoke(state)
    
#     # Print the chatbot's response
#     print("Chatbot:", response['messages'][-1].content)  # Assuming the last message is from the AI

In [7]:
input = {
    "messages": [HumanMessage(content="What is the current weather in Bangalore?")]
}

events = app.stream(input, stream_mode='values')
for event in events:
    print(event)

{'messages': [HumanMessage(content='What is the current weather in Bangalore?', additional_kwargs={}, response_metadata={}, id='9badaadb-b060-433a-9194-4967a4430c57')]}
{'messages': [HumanMessage(content='What is the current weather in Bangalore?', additional_kwargs={}, response_metadata={}, id='9badaadb-b060-433a-9194-4967a4430c57'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ROXezDpteLrXuhug88FD6NeP', 'function': {'arguments': '{"query":"current weather in Bangalore"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BhsS2x3HL6WamFELZ2Ct

In [8]:
input = {
    "messages": [HumanMessage(content="What is the current weather in Bangalore?")]
}

events = app.stream(input, stream_mode='updates')
for event in events:
    print(event)

{'chat_bot': {'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_koKZuJj1pNAH2oN6Y2Rsx2X5', 'function': {'arguments': '{"query":"current weather in Bangalore"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BhsZWyHXflnTIDlw8UTEaN1Lh407c', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9e6373fb-657c-48bc-bb4b-e0903a817f42-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Bangalore'}, 'id': 'call_koKZuJj1pNAH2oN6Y2Rsx2X5', 'type': 'tool_call'}], usage_me

In [14]:
input = {
    "messages": [HumanMessage(content="How are you ?")]
}

events = app.astream_events(input, stream_mode='updates')
async for event in events:
    if event['event'] == 'on_chat_model_stream':
        print(event['data']['chunk'].content, end='')

I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?